# 이분 탐색 (Binary Search)

## 나무 자르기
- 문제 출처: [백준 2805번](https://www.acmicpc.net/problem/2805)

`-` 이진 탐색으로 해결 가능할 듯 하다 ---> 처음 푸는 유형이었음

`-` 나무를 높이 순서로 오름차순 정렬한 후 $0$과 최대 나무 높이를 중간으로 나눠서 기준치를 넘는지 안넘는지를 판단함

`-` 목표는 나무를 $x$미터까지 잘랐을 땐 기준치 미달인데 $x+1$미터를 자르니 기준치를 넘기는 $x+1$을 찾는 것

`-` 여러번 틀려서 디버깅 해보니 나무를 반으로 나누는데 최댓값은 나무의 최대 높이로 하고 최솟값은 나무의 최소 높이로 설정했었다

`-` 위에 처럼하면 잘못됐는데 왜냐하면 이 문제의 해가 나무의 최소 높이보다 더 낮은 높이를 절단해야 하면 답이 나오지 않기 때문이다

`-` 예시로 나무가 $3$개 있고 높이가 각각 $10, 20, 30$이라 하고 필요한 나무는 $10$미터이라 하자

`-` 이 경우는 잘못된 코드로 풀어도 가능함 ---> 왜냐면 정답이 $20$미터인데 $20$은 $10$과 $30$ 사이 숫자이므로

`-` 하지만 필요한 나무가 $60$미터라면 정답은 $0$미터여야 하지만 틀린 코드는 최솟값 $10$과 최댓값 $30$ 사이를 이분 탐색하므로 $0$이 나올 수가 없음

In [77]:
def trees_sum(h):  # 잘라서 나온 나무 높이의 합 구하기
    _sum = 0
    for i in range(N):
        if x[i] > h:
            _sum += x[i] - h
    return _sum


def cut(trees):
    left = 0  # 잘못된 코드는 left = trees[0], 나머지는 동일함
    right = trees[-1]
    mid = (left + right) // 2
    sum_ = trees_sum(mid)
    while True:
        if right - left < 2:
            break    
        if sum_ >= M:    
            left = mid
            mid = (left + right) // 2
            sum_ = trees_sum(mid)  
        else:
            right = mid
            mid = (left + right) // 2
            sum_ = trees_sum(mid)
    return mid


N, M = map(int, input().split())
x = list(map(int, input().split()))
x.sort()
print(cut(x))

# input
# 4 7
# 20 15 10 17

 4 7
 20 15 10 17


15


## 수 찾기

- 문제 출처: [백준 1920번](https://www.acmicpc.net/problem/1920)

`-` 이분 탐색 문제이다

`-` `list.index`를 사용해 접근하면 시간초과가 발생한다

```python
n = int(input())
nums = list(map(int, input().split()))
nums.sort()
m = int(input())
nums2 = list(map(int, input().split()))
for num in nums2:
    left = nums[0]
    right = nums[-1]
    mid = nums[n // 2] 
    w = 0
    while nums.index(right) - nums.index(left) >= 2:
        if num == mid or num == left or num == right:
            w = 1
            break
        if num >= mid:
            left = nums[nums.index(mid)]
            mid = nums[(nums.index(left) + nums.index(right)) // 2]
        else:
            right = nums[nums.index(mid)]
            mid = nums[(nums.index(left) + nums.index(right)) // 2]
    if num == mid or num == left or num == right:
        w = 1
    print(w)
```

`-` 생각해보니 위와 같이 하지 않고도 `nums`의 인덱스에 접근할 수 있다

`-` 탐색구간을 `nums`가 아닌 $k = 1 \sim n$까지 탐색하고 `nums[k]`를 사용해 인덱싱하면 된다

In [46]:
n = int(input())
nums = list(map(int, input().split()))
nums.sort()
m = int(input())
nums2 = list(map(int, input().split()))
for num in nums2:
    left = 0
    right = n - 1
    mid = (left + right) // 2 
    while right - left >= 2:
        if num >= nums[mid]:
            left = mid 
            mid = (left + right) // 2 
        else:
            right = mid
            mid = (left + right) // 2 
    if nums[mid] == num or nums[right] == num:  # mid와 left는 동일해진다, right = mid + 1
        print(1)
    else:
        print(0)

# input
# 5
# 4 1 5 2 3
# 5
# 1 3 7 9 5

 5
 4 1 5 2 3
 5
 1 3 7 9 5


1
1
0
0
1


## 랜선 자르기

- 문제 출처: [백준 1654번](https://www.acmicpc.net/problem/1654)

`-` 나무 자르기 문제와 비슷한 유형이다

`-` 임의의 랜선길이를 $s$라고 하자

`-` 각 랜선들에 대해 $s$로 나눈 몫이 잘라서 나온 랜선의 개수이고 이들의 합이 총 개수이다

`-` $k$개의 랜선이 필요하다고 하자

`-` 만약 $s$로 나눈 몫들의 합이 $k$라면?

`-` 일단 랜선의 개수 조건은 충족시켰으나 동일한 $k$에 대해 $s$를 최대화시켜야 한다 

`-` 만약 $s$로 나눈 몫들의 합이 $k$보다 작았는데 $s-1$로 나눈 몫들의 합이 $k$라면?

`-` 그 때의 $s$(임계값)가 바로 최대 랜선 길이이다

In [1]:
def total(data, s):  # 자르고 나온 랜선 개수 총합
    return sum(list(map(lambda x: x // s, data)))


k, n = map(int, input().split())
data = [int(input()) for _ in range(k)]
low = 1
high = max(data)  # min(data)로 하면 틀림, 1000, 1000, 1 ---> 3조각 낸다고 생각하자 ---> 답은 1? ㅋㅋㅋ
# 만약 최소길이 랜선을 이용하지 않으면 요구된 랜선 개수를 충족시키지 못할 때 high = min(data)로 설정해도 된다
# 하지만 그런 상황만 주어지는 것은 아니기에...
mid = (low + high) // 2
while high - low >= 2:
    if total(data, mid) < n:
        high = mid
        mid = (low + high) // 2
    else:  # 만약에 total(data, mid) == n이라면 이를 만족하는 mid에 대해서 mid값을 계속 증가킴으로써 임계값을 구한다       
        low = mid
        mid = (low + high) // 2
# 아래 부분을 제외하면 틀린다 ---> why?
# 일단 위의 코드 결과는 low = mid, right = mid + 1로 나온다 (while문 조건에 의해)
# 만약 low가 임계값 - 1이라고 하자 ---> 정답은 low + 1이다
# mid는 low보다 크기에 else문이 계속하여 실행된다
# 코드를 실행하다 결과적으로 high = low + 1이 되면서 while문이 끝난다!
# 그런데 low = mid = 임계값 - 1인 상황이다
# 그러므로 위와 같은 상황을 처리하기 위해 아래 부분이 필요
if total(data, high) == total(data, mid): 
    print(high)
else:
    print(mid)

# input
# 4 11
# 802
# 743
# 457
# 539

 4 11
 802
 743
 457
 539


200


## K번째 수

- 문제 출처: [백준 1300번](https://www.acmicpc.net/problem/1300)

`-` 의외로 이분 탐색으로 풀 수 있는 놀라운 문제라고 한다

`-` 단순히 $N^2$개의 원소를 정렬한 후 $k$번째 원소를 찾는 건 시간 초과이다

`-` 반대각선을 긋고 어떻게든 구상을 해볼려 했는데 잘 안됐다

`-` 근데 생각해보니 $(x,x)$의 원소보다 작거나 같은 원소의 개수를 $O(N)$에 계산할 수 있다

`-` $(i,j)$의 원소는 $i\times j$이므로 $i$행에서 열번호 $j$가 $\left\lfloor\dfrac{x^2}{i}\right\rfloor$보다 작거나 같으면 $x^2$보다 작거나 같다

`-` 즉, $i$행의 원소 중 $\left\lfloor\dfrac{x^2}{i}\right\rfloor$개는 $(x,x)$의 원소보다 작다

`-` 이를 $1$행부터 $N$행까지 반복하면 $(x,x)$의 원소보다 작거나 같은 원소의 개수를 알 수 있다

`-` $x^2$보다 작은 원소의 개수가 $k$개 이상인 $x$ 중 가장 작은 $x$를 $x_*$라고 하자

`-` 그럼 $k$번째 원소는 $(x_* - 1)^2$보다 크고 $x_*^2$보다 작거나 같다

`-` $(x_* - 1)^2$보다 작거나 같은 원소의 개수를 $q$라고 하자

`-` $(x_* - 1)^2$보다 크고 $x_*^2$보다 작거나 같은 원소들의 집합을 $S$라고 하자

`-` $S$를 오름차순 정렬했을 때 $k-q$번째에 위치한 원소가 전체 원소 중 $k$번째 수가 된다

`-` 집합 $S$를 구성하는 원소를 구하면 끝이다

`-` $i$행에 대해 열 번호가 $\left\lfloor\dfrac{(x_*-1)^2}{i}\right\rfloor$보다 크고 $\left\lfloor\dfrac{x_*^2}{i}\right\rfloor$보다 작거나 같으면 집합 $S$에 속한다

`-` 그러한 원소의 개수는 최악의 경우 러프하게 $(2N-1)\sum\limits_{i=1}^{i=N}\dfrac{1}{i}$이고 이는 $O(N\log N)$이다

`-` $x_*$를 찾는 건 $N$개의 제곱 수에 대해 이진 탐색으로 찾으면 된다

In [36]:
def count_lower_bound(x, n):
    num = 0
    for i in range(1, n + 1):
        num += min(x**2 // i, n)
    return num


def binary_search(n, k):
    lower = 1
    upper = n
    while lower < upper:
        mid = (lower + upper) // 2
        num = count_lower_bound(mid, n)
        if num >= k:
            upper = mid
        else:
            lower = mid + 1
    return upper
    

def solution():
    N = int(input())
    k = int(input())
    x = binary_search(N, k)
    q = count_lower_bound(x - 1, N)
    elements = []
    for i in range(1, N + 1):
        start = min((x - 1)**2 // i, N) + 1
        end = min(x**2 // i, N) 
        for j in range(start, end + 1):
            e = i * j
            elements.append(e)
    elements.sort()
    answer = elements[k - q - 1]
    print(answer)


solution()

# input
# 3
# 7

 3
 7


6


`-` 질문 검색을 보니 더 쉽게 해결할 수 있는 방법이 있었다

`-` 나는 반대각선의 위치한 원소에 대해 그 원소보다 작거나 같은 원소의 개수만 셌는데 이렇게 하는게 아니었다

`-` 배열 $B$의 원소는 $1$과 $N^2$ 사이의 값을 가진다

`-` 따라서 하한을 $1$로 설정하고 상한을 $N^2$으로 설정하여 $k$번째 수가 될 수 있는 $B$의 원소에 대해 이분 탐색을 하는 것이다

`-` $O(N)$에 임의의 원소보다 작은 원소의 개수를 구할 수 있다 (기준이 $x^2$이 아니게 될 뿐이다)

`-` 따라서 이분 탐색을 하면서 `mid`보다 작은 원소의 개수가 $k-1$개가 되도록 하는 `mid` 중 최솟값을 찾으면 된다

`-` 그럼 $k$번째 수는 `mid`가 된다

`-` 이분 탐색 중 `mid`보다 작은 원소의 개수가 $k-1$이라고 탐색을 멈추면 안되고 범위를 더 줄여서 탐색해야 한다

`-` 왜냐하면 `mid`가 $N$보다 큰 소수일 수 있기 때문이다 (배열 $B$에 속하지 않는다)

`-` 최종적으로 탐색할 공간이 없게 될 때의 `mid`가 정답이 된다